In [96]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# loan level data
loanlevel_file = r"/Users/xiaoyu/Downloads/fsq-21-1gr-boe-aug-21.csv"

In [3]:
import pandas as pd
import toolz as tz
from lenses import lens

In [4]:
loanAssetDF = pd.read_csv(loanlevel_file)

/var/folders/pb/0lsr_g9s0x340sd74jhjdhj40000gn/T/ipykernel_63319/531353068.py:1: DtypeWarning: Columns (20,27,72,106,107,117,119,136) have mixed types. Specify dtype option on import or set low_memory=False.
  loanAssetDF = pd.read_csv(loanlevel_file)


In [5]:
from dateparser import *
from dateutil import relativedelta

In [6]:
def converQtoDate(y):
    d = y.replace("Q1","03-30").replace("Q2","06-30").replace("Q3","09-30").replace("Q4","12-31")
    return parse(d)

In [7]:
def mapping(x:dict):
    freqMapping = {
        1:"Monthly"
    }
    repayMapping = {
        1:"Level"
        ,2:"Level"
        ,6:"I_P"
    }
    statusMapping = {
        1:"Current",
        2:"Defaulted",
        4:"Defaulted"
    }
    
    def converQtoDate(y):
        d = y.replace("Q1","03-30").replace("Q2","06-30").replace("Q3","09-30").replace("Q4","12-31")
        return parse(d)
    
    def calDateAndTerm(s,oTerm,cutoffDate):
        sd = converQtoDate(s)
        cutoffDate = parse(cutoffDate)
        age = relativedelta.relativedelta(cutoffDate,sd)
        monthsPast = age.years*12+age.months
        
        return (sd.strftime("%Y-%m-%d")
                , oTerm - monthsPast)
    
    originDate,remainTerm = calDateAndTerm(x["AR55"],x['AR61'],x['AR1'])
    
    
    return ["Mortgage"
              ,{"originBalance": x['AR66']
                ,"originRate": ["fix",x['AR109']]
                ,"originTerm": x['AR61']
                ,"freq": freqMapping[x['AR70']]
                ,"type": repayMapping[x['AR72']]
                ,"originDate": originDate}
              ,{"currentBalance": x['AR67']
                ,"currentRate": x['AR109']
                ,"remainTerm": remainTerm
                ,"status": statusMapping[x['AR166']]
               }
           ]

In [40]:
from absbox import API,EnginePath

localAPI = API(EnginePath.LOCAL,check=False)

Connecting engine server -> http://localhost:8081

✅Connected, local lib:0.28.7, server:0.42.1

In [41]:
def mergeAssets(xs):
    cbals = lens.Each()[2]['currentBalance'].collect()(xs)
    crates = lens.Each()[2]['currentRate'].collect()(xs)
    obals = lens.Each()[1]['originBalance'].collect()(xs)
    orates = lens.Each()[1]['originRate'][1].collect()(xs)

    ot = xs[0][1]['originTerm']
    rt = xs[0][2]['remainTerm']
    sd = xs[0][1]['originDate']
    st = xs[0][2]['status']
    freq = xs[0][1]['freq']
    t = xs[0][1]['type']
    totalCurBal = sum(cbals)
    totalOriBal = sum(obals)
    
    if totalCurBal == 0:
        return None
    avgRate = sum([ x*y for x,y in zip(cbals,crates)])/totalCurBal
    avgOriRate = sum([ x*y for x,y in zip(obals,orates)])/totalOriBal

    return ['Mortgage',
               {'originBalance': totalOriBal,
                'originRate': ['fix', 0.01],
                'originTerm': ot,
                'freq': freq,
                'type': t,
                'originDate': sd},
               {'currentBalance': totalCurBal,
                'currentRate': avgRate,
                'remainTerm': rt,
                'status': st}]

def consolAsset(ast_list):
    return tz.pipe(tz.groupby(lambda x:(x[1]['originTerm'],x[1]['freq'],x[1]['type'],x[1]['originDate'],x[2]['remainTerm'],x[2]['status']), ast_list)
                   ,lambda m: tz.valmap(lambda x: mergeAssets(x), m)
                   ,lambda m: [ _ for _ in m.values() if _ and _[2]['status']!="Defaulted"]
                   )

loanAssetInMap = loanAssetDF.to_dict('records')

In [53]:
myPool = {"assets": [ mapping(_) for _ in loanAssetInMap  ][:1500], "cutoffDate":"2021-08-31" }

In [54]:
len(myPool['assets'])

1500

In [66]:
%%timeit

rp = localAPI.runPool(myPool,read=True)

3.52 s ± 39.9 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [65]:
%%timeit

rp = localAPI.runPool(myPool,read=True)

2.23 s ± 30.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [64]:
(3.5 - 2.3)/3.5

0.3428571428571429